In [1]:
# !pip install -qU pip wheel
# !pip install -q -r ../requirements.txt
import sys
sys.path.append('..')
from ballitoreproject import *
from ballitoreproject.networks import *

In [8]:
G=get_correspondence_network()

In [17]:
plot_network(G, 
    edge_width='num_letters',
    node_color='is_family',
    edge_color='relationship',
    node_colormap='Pastel1'
)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

In [4]:
list(G.nodes(data=True))

[('Richard Shackleton', {'generation': 1, 'gender': 'M', 'is_family': True}),
 ('Elizabeth Fuller Shackleton',
  {'generation': 1, 'gender': 'F', 'is_family': True}),
 ('Elizabeth Carleton Shackleton',
  {'generation': 1, 'gender': 'F', 'is_family': True}),
 ('Margaret Grubb', {'generation': 2, 'gender': 'F', 'is_family': True}),
 ('Deborah Chandlee', {'generation': 2, 'gender': 'F', 'is_family': True}),
 ('Mary Leadbeater', {'generation': 2, 'gender': 'F', 'is_family': True}),
 ('Sarah Shackleton', {'generation': 2, 'gender': 'F', 'is_family': True}),
 ('Abraham Shackleton', {'generation': 2, 'gender': 'M', 'is_family': True}),
 ('William Leadbeater', {'generation': 2, 'gender': 'M', 'is_family': True}),
 ('Sarah Barrington', {'generation': 3, 'gender': 'F', 'is_family': True}),
 ('Edward Barrington', {'generation': 3, 'gender': 'M', 'is_family': True}),
 ('Margaret Davis', {'generation': 4, 'gender': 'F', 'is_family': True}),
 ('Abigail Wright', {'is_family': False}),
 ('Sarah Leadbe

In [5]:
list(G.edges(data=True))

[('Richard Shackleton',
  'Elizabeth Fuller Shackleton',
  {'relationship': 'married'}),
 ('Richard Shackleton',
  'Elizabeth Carleton Shackleton',
  {'relationship': 'married', 'num_letters': 14, 'num_words': 2599}),
 ('Richard Shackleton', 'Margaret Grubb', {'relationship': 'parented'}),
 ('Richard Shackleton', 'Deborah Chandlee', {'relationship': 'parented'}),
 ('Richard Shackleton', 'Mary Leadbeater', {'relationship': 'parented'}),
 ('Richard Shackleton', 'Sarah Shackleton', {'relationship': 'parented'}),
 ('Richard Shackleton', 'Abraham Shackleton', {'relationship': 'parented'}),
 ('Richard Shackleton',
  'Deborah Christy',
  {'relationship': 'wrote', 'num_letters': 96, 'num_words': 34253}),
 ('Richard Shackleton',
  'Elizabeth Pim Pike',
  {'relationship': 'wrote', 'num_letters': 45, 'num_words': 5771}),
 ('Richard Shackleton',
  'James Abell',
  {'relationship': 'wrote', 'num_letters': 31, 'num_words': 4249}),
 ('Richard Shackleton',
  'Joseph Jacob',
  {'relationship': 'wrote',

In [6]:
cyto = ipycytoscape.CytoscapeWidget()
cyto.graph.add_graph_from_networkx(G,directed=True)

In [7]:
cyto.graph.nodes

[Node(data={'generation': 1, 'gender': 'M', 'is_family': True, 'id': 'Richard Shackleton'}, position={}),
 Node(data={'generation': 1, 'gender': 'F', 'is_family': True, 'id': 'Elizabeth Fuller Shackleton'}, position={}),
 Node(data={'generation': 1, 'gender': 'F', 'is_family': True, 'id': 'Elizabeth Carleton Shackleton'}, position={}),
 Node(data={'generation': 2, 'gender': 'F', 'is_family': True, 'id': 'Margaret Grubb'}, position={}),
 Node(data={'generation': 2, 'gender': 'F', 'is_family': True, 'id': 'Deborah Chandlee'}, position={}),
 Node(data={'generation': 2, 'gender': 'F', 'is_family': True, 'id': 'Mary Leadbeater'}, position={}),
 Node(data={'generation': 2, 'gender': 'F', 'is_family': True, 'id': 'Sarah Shackleton'}, position={}),
 Node(data={'generation': 2, 'gender': 'M', 'is_family': True, 'id': 'Abraham Shackleton'}, position={}),
 Node(data={'generation': 2, 'gender': 'M', 'is_family': True, 'id': 'William Leadbeater'}, position={}),
 Node(data={'generation': 3, 'gender'